In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.colors as cx
import matplotlib.cm as cm
import os
import seaborn as sns
import umap
import csv
import warnings
from collections import OrderedDict
from collections import defaultdict
from scipy import signal, stats
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import Patch
from scipy.interpolate import interp1d
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams.update({'figure.max_open_warning': 0})
warnings.simplefilter("ignore")

%run grooming_functions.ipynb
%matplotlib inline

# for figure styling and saving
sns.set()
sns.set_style('ticks')
out_path = '/media/turritopsis/katie/grooming/t1-grooming'

In [ ]:
# define functions 
def get_url(flyid, filename):
    url_prefix = 'http://128.95.10.233:5000'
    session, _, folder = flyid.partition('_')
    url = '{}/#{}/Fly {}/{}'.format(url_prefix, session, folder, filename)
    url = url.replace(' ', '%20')
    return url

# root mean square error between two time series
def rmse(predictions, targets):
    error = np.sqrt(np.mean((predictions-targets)**2))
    return error

def get_cycles(data, angle_vars, align_to = None, get_all_cycles = False, min_length = 0, dist = 20, height = None, npeaks = 1):
    
    cycle_dict = dict()
    length_dict = dict()
    bout_numbers = np.unique(data.behavior_bout.astype(int))

    for i in range(len(angle_vars)): 

        all_cycles = []
        all_lengths = []

        for j in range(len(bout_numbers)):

            bout = data[data.behavior_bout == bout_numbers[j]]
            if align_to is None:
                bout_angs = np.array(bout[angle_vars[i]])
            else: 
                bout_angs = np.array(bout[align_to])
                
            if len(bout_angs) < min_length:
                continue
                
            idxs, props = signal.find_peaks(bout_angs, distance = dist, height = height)
            peaks = bout_angs[idxs]
            
            if len(peaks) < 1:
                #problem
                continue

            cycles = []
            lengths = []
            bout_angs = np.array(bout[angle_vars[i]])
            for k in range(len(peaks)-npeaks):
                cycle = np.zeros(600)
                cycle[:] = np.nan
                period = bout_angs[idxs[k]:idxs[k+npeaks]]
                cycle[:len(period)] = period
                cycles.append(cycle)
                lengths.append(len(period))
            
            if not get_all_cycles:
                all_cycles.append(cycles[len(cycles)//2])
                all_lengths.append(lengths[len(lengths)//2])
            else:
                all_cycles.extend(cycles)
                all_lengths.extend(lengths)

        cycle_dict[angle_vars[i]] = all_cycles
        length_dict[angle_vars[i]] = all_lengths
       
    return cycle_dict, length_dict

def get_average(all_cycles, all_lengths, npoints, align_start = True, stretch = False):

    xs_all = np.linspace(0, np.nanmin(all_lengths)-1, num = npoints, endpoint=True)
    if stretch:
        xs_all = np.linspace(0, 1, num = npoints, endpoint=True)
    avg_data = np.zeros([len(all_cycles), len(xs_all)])

    for j in range(len(all_cycles)):
        cycle = np.array(all_cycles[j][:all_lengths[j]])
        xs = np.arange(len(cycle))
        if stretch: 
            xs = xs / (len(cycle)-1)
        f = interp1d(xs, cycle,'cubic')
        f_int = f(xs_all)
        avg_data[j, :] = f_int
    avg = np.mean(avg_data, axis = 0) 
    
    return xs_all, avg, avg_data


def plot_cycles(cycle_dict, length_dict, angle_vars, average = False, align_start = True, stretch = False, title_extension = '', fps = 300.0, legend = False, cmap = 'plasma'):
    
    for j in range(len(angle_vars)): 

        all_cycles = cycle_dict[angle_vars[j]]
        all_lengths = length_dict[angle_vars[j]]
        
        if len(all_lengths) == 0: 
            continue

        fig = plt.figure(figsize = (8,4))
        plt.title('{} {} angles {}'.format(angle_vars[j].split('_')[0][:2], titles[angle_vars[j][2:]], title_extension), fontsize = 14)     
        plt.xlabel('time (seconds)', fontsize = 14)
        plt.ylabel('angle (deg)', fontsize = 14) 
        
        min_len = np.percentile(all_lengths,5)
        max_len = np.percentile(all_lengths,95)
        cmap = plt.get_cmap(cmap)
        cnorm  = cx.Normalize(vmin=min_len, vmax=max_len)
        # cnorm  = cx.Normalize(vmin=np.nanmin(all_lengths), vmax=np.nanmax(all_lengths))
        scalar_map = cm.ScalarMappable(norm=cnorm, cmap=cmap)

        for k in range(len(all_cycles)):
        
            ax = fig.add_subplot(111)
            cycle = all_cycles[k][:all_lengths[k]]
            xs = np.arange(len(cycle))
            if align_start:
                cycle = cycle - cycle[0]
                cycle_centered = np.zeros(600)
                cycle_centered[:] = np.nan
                cycle_centered[:all_lengths[k]] = cycle
                all_cycles[k] = cycle_centered
                
            if stretch: 
                xs = np.arange(len(cycle)) / (len(cycle)-1)
                ax.set_xticks([])
                ax.set_xticklabels([])
                
            if len(cycle) >= min_len and len(cycle) <= max_len:
                ax.plot(xs / fps, cycle, linewidth=1, color = scalar_map.to_rgba(all_lengths[k]))

        #if not stretch: 
            #avg_cycle = np.nanmean(all_cycles, axis = 0)
            #avg_cycle = avg_cycle[:max(all_lengths)]
            #xs = np.arange(max(all_lengths))
            #xs_max = xs[xs < np.percentile(xs, 80)]
            #plt.plot(xs_max / fps, avg_cycle[:len(xs_max)], label = 'average', color = 'k')
            #plt.xlim([0, np.percentile(xs, 80)/fps])
            
        if average:
            npoints = 51
            xs_all, avg, avg_data = get_average(all_cycles, all_lengths, npoints, align_start = align_start, stretch = stretch)
            plt.plot(xs_all / fps, avg, 'k')
            
        if legend:
            plt.legend(fontsize = 12, loc=(1.02,0.2)) 
            
        sns.despine()
        plt.show() 
        
def get_phases(cycle_dict, length_dict, npoints=None):
    
    angle_vars = list(cycle_dict.keys())
    phase_dict = dict()
    angle_dict = dict()
    
    for i in range(len(angle_vars)):
    
        all_cycles = cycle_dict[angle_vars[i]]
        if npoints is not None: 
            all_lengths = [npoints]*len(all_cycles)
        else:
            all_lengths = length_dict[angle_vars[i]]
        all_phases = [] 
        all_angles = []

        min_len = np.percentile(all_lengths,5)
        max_len = np.percentile(all_lengths,95)
        for j in range(len(all_cycles)):
            cycle = all_cycles[j][:all_lengths[j]]
            if len(cycle) >= min_len and len(cycle) <= max_len:
                xs = np.arange(all_lengths[j]) / (all_lengths[j]-1)
                ht = signal.hilbert(cycle)
                phases = np.angle(ht)
                all_phases.append(phases)
                all_angles.append(cycle)
            
        phase_dict[angle_vars[i]] = all_phases
        angle_dict[angle_vars[i]] = all_angles
    
    return phase_dict, angle_dict

def plot_phases(phase_dict, angle_dict, fps = 300.0, cmap = 'plasma'):
    
    angle_vars = list(phase_dict.keys())
    for i in range(len(angle_vars)):
        
        all_phases = phase_dict[angle_vars[i]]
        all_angles = angle_dict[angle_vars[i]]
        
        cmap = plt.get_cmap(cmap)
        cnorm  = cx.Normalize(vmin=len(min(all_phases, key=len)), vmax=len(max(all_phases, key=len)))
        scalar_map = cm.ScalarMappable(norm=cnorm, cmap=cmap)
        
        figure = plt.figure()
        ax = plt.gca()
        ax.set_title('{}'.format(angle_vars[i]), fontsize = 14)
        ax.set_xlabel('time (seconds)', fontsize = 14)
        ax.set_ylabel('phase (rad)', fontsize = 14)
        ax.set_ylim([-np.pi, np.pi])
        ax.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
        ax.set_yticklabels(['-pi', '-pi/2', '0', 'pi/2', 'pi'])
        
        for j in range(len(all_phases)):        
            xs = np.arange(len(all_phases[j])) / fps
            plt.plot(xs, all_phases[j], color = scalar_map.to_rgba(len(all_phases[j])))
        
        plt.show()
        
def plot_phases_and_angles(phase_dict, angle_dict, align_to, cmap = 'plasma'):
    
    angle_vars = list(phase_dict.keys())
    for i in range(len(angle_vars)):
        
        all_phases = phase_dict[align_to]
        all_angles = angle_dict[angle_vars[i]]
                
        cmap = plt.get_cmap(cmap)
        cnorm  = cx.Normalize(vmin=len(min(all_phases, key=len)), vmax=len(max(all_phases, key=len)))
        scalar_map = cm.ScalarMappable(norm=cnorm, cmap=cmap)
        
        figure = plt.figure()
        ax = plt.gca()
        ax.set_title('{} angles'.format(angle_vars[i]), fontsize = 14)
        ax.set_xlabel('phase (rad)', fontsize = 14)
        ax.set_ylabel('angle (deg)', fontsize = 14)
        ax.set_xlim([-np.pi, np.pi])
        ax.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
        ax.set_xticklabels(['$-\pi$', '-$\pi$/2', '0', '$\pi$/2', '$pi$'])
        # ax.set_ylim([-55, 55])
        
        for j in range(len(all_phases)):
            plt.scatter(all_phases[j], all_angles[j], s = 2, color = scalar_map.to_rgba(len(all_phases[j])), alpha = 0.7)
            
        plt.show()

In [ ]:
def subplot_cycles(cycle_dict, length_dict, angle_vars, average = False, align_start = True, stretch = False, title_extension = '', fps = 300.0, legend = False, cmap = 'plasma'):
    
    angle_vars_u = np.unique([x[2:] for x in angle_vars])
    legs = np.unique([x[:2] for x in angle_vars])
    
    for j in range(len(angle_vars_u)):
        
        fig = plt.figure(figsize = (12,3))
        for i in range(len(legs)):     

            all_cycles = cycle_dict[legs[i] + angle_vars_u[j]]
            all_lengths = length_dict[legs[i] + angle_vars_u[j]]

            if len(all_lengths) == 0: 
                continue
 
            cmap = plt.get_cmap(cmap)
            cnorm  = cx.Normalize(vmin=np.nanmin(all_lengths), vmax=np.nanmax(all_lengths))
            scalar_map = cm.ScalarMappable(norm=cnorm, cmap=cmap)

            for k in range(len(all_cycles)):

                ax = fig.add_subplot(1,2,i+1)
                ax.set_title('{} {} angles {}'.format(legs[i], titles[angle_vars_u[j]], title_extension), fontsize = 14)     
                ax.set_xlabel('time (seconds)', fontsize = 14)
                ax.set_ylabel('angle (deg)', fontsize = 14) 
                cycle = all_cycles[k][:all_lengths[k]]
                xs = np.arange(len(cycle))
                if align_start:
                    cycle = cycle - cycle[0]
                    cycle_centered = np.zeros(600)
                    cycle_centered[:] = np.nan
                    cycle_centered[:all_lengths[k]] = cycle
                    all_cycles[k] = cycle_centered

                if stretch: 
                    xs = np.arange(len(cycle)) / (len(cycle)-1)
                    ax.set_xticks([])
                    ax.set_xticklabels([])

                ax.plot(xs / fps, cycle, linewidth=1, color = scalar_map.to_rgba(all_lengths[k]))

            #if not stretch: 
            #    avg_cycle = np.nanmean(all_cycles, axis = 0)
            #    avg_cycle = avg_cycle[:max(all_lengths)]
            #    xs = np.arange(max(all_lengths))
            #    xs_max = xs[xs < np.percentile(xs, 80)]
            #    plt.plot(xs_max / fps, avg_cycle[:len(xs_max)], label = 'average', color = 'k')
            #    plt.xlim([0, np.percentile(xs, 80)/fps])
            
            if average:
                npoints = 51
                xs_all, avg, avg_data = get_average(all_cycles, all_lengths, npoints, align_start = align_start, stretch = stretch)
                plt.plot(xs_all / fps, avg, 'k')

            if legend:
                plt.legend(fontsize = 12, loc=(1.02,0.2)) 
            
        sns.despine()
        plt.show() 
        
def subplot_phases(phase_dict, angle_dict, fps = 300.0, cmap = 'plasma'):
    
    angle_vars = list(phase_dict.keys())
    angle_vars_u = np.unique([x[2:] for x in angle_vars])
    legs = np.unique([x[:2] for x in angle_vars])
    
    for i in range(len(angle_vars_u)):
        
        fig = plt.figure(figsize = (12, 3))
        for k in range(len(legs)):
        
            all_phases = phase_dict[legs[k] + angle_vars_u[i]]
            all_angles = angle_dict[legs[k] + angle_vars_u[i]]
            
            cmap = plt.get_cmap(cmap)
            cnorm  = cx.Normalize(vmin=len(min(all_phases, key=len)), vmax=len(max(all_phases, key=len)))
            scalar_map = cm.ScalarMappable(norm=cnorm, cmap=cmap)
            
            ax = fig.add_subplot(1, 2, k+1)
            ax.set_title('{}'.format(legs[k] + angle_vars_u[i]), fontsize = 14)
            ax.set_xlabel('time (seconds)', fontsize = 14)
            ax.set_ylabel('phase (rad)', fontsize = 14)
            ax.set_ylim([-np.pi, np.pi])
            ax.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
            ax.set_yticklabels(['-pi', '-pi/2', '0', 'pi/2', 'pi'])

            for j in range(len(all_phases)):        
                xs = np.arange(len(all_phases[j])) / fps
                ax.plot(xs, all_phases[j], color = scalar_map.to_rgba(len(all_phases[j])))
        
        plt.subplots_adjust(wspace = 0.3)
        plt.show()
        
def subplot_phases_and_angles(phase_dict, angle_dict, align_to, cmap = 'plasma'):
    
    angle_vars = list(phase_dict.keys())
    angle_vars_u = np.unique([x[2:] for x in angle_vars])
    legs = np.unique([x[:2] for x in angle_vars])
    
    for i in range(len(angle_vars_u)):
        
        fig = plt.figure(figsize = (12,3))
        for k in range(len(legs)):
        
            all_phases = phase_dict[align_to]
            all_angles = angle_dict[legs[k] + angle_vars_u[i]]

            cmap = plt.get_cmap(cmap)
            cnorm  = cx.Normalize(vmin=len(min(all_phases, key=len)), vmax=len(max(all_phases, key=len)))
            scalar_map = cm.ScalarMappable(norm=cnorm, cmap=cmap)

            ax = fig.add_subplot(1,2,k+1)
            ax.set_title('{} angles'.format(legs[k] + angle_vars_u[i]), fontsize = 14)
            ax.set_xlabel('phase (rad)', fontsize = 14)
            ax.set_ylabel('angle (deg)', fontsize = 14)
            ax.set_xlim([-np.pi, np.pi])
            ax.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
            ax.set_xticklabels(['$-\pi$', '-$\pi$/2', '0', '$\pi$/2', '$pi$'])
            ax.set_ylim([-55, 55])

            for j in range(len(all_phases)):       
                ax.scatter(all_phases[j], all_angles[j], s=2, color = scalar_map.to_rgba(len(all_phases[j])))
    
        plt.subplots_adjust(wspace = 0.2)
        plt.show()

In [ ]:
# all functions for the heatmaps
def get_angles_aligned(phase_dict, angle_dict, align_to, margin=0.1):
    angle_vars = np.array(list(phase_dict.keys()))
    phases = concat_all(phase_dict[align_to])
    mask = (phases > -margin) & (phases < margin)
    angles_aligned_dict = dict()
    for j in range(len(angle_vars)): 
        angles = concat_all(angle_dict[angle_vars[j]])
        angles = angles[mask]
        angles_aligned_dict[angle_vars[j]] = np.nanmean(angles)
    return angles_aligned_dict

def concat_all(array):
    extended_array = []
    for j in range(len(array)):
        extended_array.extend(array[j])
    extended_array = np.array(extended_array)
    return extended_array

def plot_phase_counts(phase_dict, align_to, nbins = 15, absolute = False):
    angle_vars = np.array(list(phase_dict.keys()))
    for j in range(len(angle_vars)):
        phases = concat_all(phase_dict[angle_vars[j]])
        if absolute:
            phases = np.abs(phases)
        counts, edges = np.histogram(phases, bins = nbins)
        fig = plt.figure()
        ax = plt.gca()
        ax.set_xlabel('phase', fontsize = 14)
        ax.set_ylabel('counts', fontsize = 14)
        ax.set_title('phase offset of {} relative to {}'.format(angle_vars[j], align_to), fontsize = 14)
        ax.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
        ax.set_xticklabels(['$-\pi$', '-$\pi$/2', '0', '$\pi$/2', '$pi$'])
        ax.hist(phases, bins = nbins, color = 'k')
        plt.show()
        
def get_phase_matrices(phase_dict, angle_dict, align_to, bins):
    
    angle_vars = np.array(list(phase_dict.keys()))
    counts_matrix = np.zeros([len(angle_vars), bins])
    angles_matrix = np.zeros([len(angle_vars), bins])
    phases_matrix = np.zeros([len(angle_vars), bins])
    phases_aligned = concat_all(phase_dict[align_to])
    counts_aligned, edges_aligned = np.histogram(phases_aligned, bins = bins)
    for j in range(len(angle_vars)):
        phases = concat_all(phase_dict[angle_vars[j]])
        angles = concat_all(angle_dict[angle_vars[j]])
        counts, edges = np.histogram(phases, bins = bins)
        angle_bins = np.digitize(phases_aligned, edges_aligned[1:-1])
        phase_bins = np.digitize(phases_aligned, edges_aligned[1:-1])
        angle_averages = np.zeros(bins)
        phase_averages = np.zeros(bins)
        for k in range(len(angle_averages)):
            angle_mask = np.where(angle_bins == k)
            phase_mask = np.where(phase_bins == k)
            angle_averages[k] = np.nanmean(angles[angle_mask])
            phase_averages[k] = np.nanmean(phases[phase_mask])
        counts_matrix[j, :] = counts
        angles_matrix[j, :] = angle_averages
        phases_matrix[j, :] = phase_averages
        
    return counts_matrix, angles_matrix, phases_matrix

# sorts angles by type (usually alphabetical)
def sort_angle_names(angle_vars, matrix):
    angle_vars = sorted(angle_vars)
    angle_indexes = np.arange(0, len(angle_vars), 1)
    angle_types = ['_abduct', '_rot', '_flex']
    indexes = []
    sorted_angles = []
    for j in range(len(angle_types)):
        angs = [x for x in angle_vars if angle_types[j] in x]
        idxs = [x for x in angle_indexes if angle_types[j] in angle_vars[x]]
        sorted_angles.extend(angs)
        indexes.extend(idxs)
        
    sorted_matrix = np.zeros(matrix.shape)
    for j in range(matrix.shape[0]):
        sorted_matrix[j, :] = matrix[indexes[j], :]
        
    return sorted_angles, sorted_matrix

def plot_counts_matrix(matrix, angle_vars, align_to, cmap = 'gist_heat', norm=True, sort_angles=False, title_extension = ''):

    if norm:
        matrix = matrix / np.max(matrix)
    
    if sort_angles:
        angle_vars, matrix = sort_angle_names(angle_vars, matrix)
        
    sns.set()
    sns.set_style('ticks')
    fig = plt.figure(figsize = (6,6))
    ax = plt.gca()
    ax.set_title('density of phase offsets {}'.format(title_extension), fontsize = 14)
    ax.set_xlabel('phase of {} (radians)'.format(align_to), fontsize = 14)
    ax.set_ylabel('joint angle', fontsize = 14)
    bins = matrix.shape[1]
    if bins % 2 == 0:
        ax.set_xticks([-0.5, bins/4-0.5, bins/2-0.5, 3*bins/4-0.5, bins-0.5])
    else:
        ax.set_xticks([-0.5, bins/4, bins/2, 3*bins/4, bins-0.5])
    ax.set_xticklabels(['-$\pi$', '-$\pi$/2', '0', '$\pi$/2', '$\pi$'])
    ax.set_yticks(np.arange(0, matrix.shape[0], 1))
    ax.set_yticklabels(angle_vars)

    im = ax.imshow(matrix, cmap = cmap)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.08)
    cbar = fig.colorbar(im, cax = cax, orientation = 'vertical')
    cbar.set_label('phase density (norm.)', fontsize = 14)
    cbar.set_clim(vmin = 0, vmax = np.amax(matrix))
    cbar.ax.tick_params(length = 3)
    plt.show()

def plot_angles_matrix(matrix, angle_vars, align_to, cmap = 'gist_heat', sort_angles=False, title_extension = ''):
        
    if sort_angles:
        angle_vars, matrix = sort_angle_names(angle_vars, matrix)
    
    sns.set()
    sns.set_style('ticks')
    fig = plt.figure(figsize = (6,6))
    ax = plt.gca()
    ax.set_title('average joint angles {}'.format(title_extension), fontsize = 14)
    ax.set_xlabel('phase of {} (radians)'.format(align_to), fontsize = 14)
    ax.set_ylabel('joint angle', fontsize = 14)
    bins = matrix.shape[1]
    if bins % 2 == 0:
        ax.set_xticks([-0.5, bins/4-0.5, bins/2-0.5, 3*bins/4-0.5, bins-0.5])
    else:
        ax.set_xticks([-0.5, bins/4, bins/2, 3*bins/4, bins-0.5])
    ax.set_xticklabels(['-$\pi$', '-$\pi$/2', '0', '$\pi$/2', '$\pi$'])
    ax.set_yticks(np.arange(0, matrix.shape[0], 1))
    ax.set_yticklabels(angle_vars)

    im = ax.imshow(matrix, cmap = cmap)
    bound = np.max([np.abs(np.min(matrix)), np.abs(np.max(matrix))])
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.08)
    cbar = fig.colorbar(im, cax = cax, orientation = 'vertical')
    cbar.set_label('joint angle (degrees)', fontsize = 14)
    cbar.set_clim(vmin = -bound, vmax = bound)
    im.set_clim(-bound, bound)
    #cbar.set_ticks(np.linspace(-np.pi, np.pi, 5))
    #cbar.set_ticklabels(['-$\pi$', '-$\pi$/2', 0, '$\pi$/2', '$\pi$'])
    cbar.ax.tick_params(length = 3)
    plt.show()
    
def plot_phases_matrix(matrix, angle_vars, align_to, cmap = 'gist_heat', sort_angles=False, title_extension = ''):
        
    if sort_angles:
        angle_vars, matrix = sort_angle_names(angle_vars, matrix)
        
    sns.set()
    sns.set_style('ticks')
    fig = plt.figure(figsize = (6,6))
    ax = plt.gca()
    ax.set_title('average phase offsets of angles {}'.format(title_extension), fontsize = 14)
    ax.set_xlabel('phase of {} (radians)'.format(align_to), fontsize = 14)
    ax.set_ylabel('joint angle', fontsize = 14)
    bins = matrix.shape[1]
    if bins % 2 == 0:
        ax.set_xticks([-0.5, bins/4-0.5, bins/2-0.5, 3*bins/4-0.5, bins-0.5])
    else:
        ax.set_xticks([-0.5, bins/4, bins/2, 3*bins/4, bins-0.5])
    ax.set_xticklabels(['-$\pi$', '-$\pi$/2', '0', '$\pi$/2', '$\pi$'])
    ax.set_yticks(np.arange(0, matrix.shape[0], 1))
    ax.set_yticklabels(angle_vars)

    im = ax.imshow(matrix, cmap = cmap)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.08)
    cbar = fig.colorbar(im, cax = cax, orientation = 'vertical')
    cbar.set_label('phase offset (radians)', fontsize = 14)
    cbar.set_clim(vmin = -np.pi, vmax = np.pi)
    cbar.set_ticks(np.linspace(-np.pi, np.pi, 5))
    cbar.set_ticklabels(['-$\pi$', '-$\pi$/2', 0, '$\pi$/2', '$\pi$'])
    cbar.ax.tick_params(length = 3)
    plt.show()

In [ ]:
# load data
behavior = 't1_grooming'
prefix = '/media/turritopsis/katie/grooming/summaries'
data_path = os.path.join(prefix, 'lines-' + behavior + '_processed.parquet')
data = pd.read_parquet(os.path.join(data_path), engine='fastparquet')

In [ ]:
fps = 300.0 # know this for this dataset

angle_types = np.array(['_BC', '_flex', '_rot', '_abduct'])
angle_names_t1 = get_angle_names(data, angle_types, only_t1 = True)
angle_names = get_angle_names(data, angle_types, only_t1 = False)

angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_rot', '_x', '_y', '_z'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] == 'L1']

bout_numbers = np.unique(np.array(data.behavior_bout))
fly_dict = get_fly_id(data, bout_numbers)
videos = get_videos(bout_numbers, data)
fly_videos = fly_to_video(data)
dif_flies = np.unique(list(fly_dict.values()))
fly_data, fly_names_sorted = data_per_fly(data)
bout_length_dict = get_bout_lengths(data)

In [ ]:
titles = {
    'A_flex': 'coxa flexion',
    'A_abduct': 'body-coxa abduction',
    'A_rot': 'coxa rotation',
    'B_flex': 'coxa-femur flexion',
    'B_rot': 'femur rotation',
    'C_flex': 'femur-tibia flexion',
    'C_rot': 'tibia rotation',
    'D_flex': 'tibia-tarsus flexion',
    '_BC': 'body-coxa abduction'
}

In [ ]:
# overlay grooming cycles from individual bouts, then determine an average grooming
# cycle for each angle (all flies, uses one oscillation per bout)
dist = 20
height = None
min_length = 0
align_to = 'R1C_flex'
legs = ['L1', 'R1']
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]
angle_titles = ['flexion', 'abduction', 'rotation']
fly_data, fly_names_sorted = data_per_fly(data)
fly_name = '1_0 5242019' 
# fly_name = fly_names_sorted[17]
fly_data = data#[data['flyid'] == fly_name]
cycle_dict, length_dict = get_cycles(fly_data, angle_vars, align_to = align_to, get_all_cycles = True, min_length = min_length, dist = dist, height = height)
plot_cycles(cycle_dict, length_dict, angle_vars, average = False, align_start = True, stretch = False)

In [ ]:
# phases and angles for all flies 
align_to = 'L1C_flex'
phase_dict, angle_dict = get_phases(cycle_dict, length_dict)
#plot_phases(phase_dict, angle_dict)
plot_phases_and_angles(phase_dict, angle_dict, align_to)

In [ ]:
# phase_heatmaps
bins = 16
align_to = 'L1C_flex'
title = ''# '(fly {})'.format(fly_name)
angle_vars = np.array(list(phase_dict.keys()))
counts_matrix, angles_matrix, phases_matrix = get_phase_matrices(phase_dict, angle_dict, align_to, bins)
plot_counts_matrix(counts_matrix, angle_vars, align_to, sort_angles = True, title_extension = title)
plot_angles_matrix(angles_matrix, angle_vars, align_to, cmap = 'twilight_shifted', sort_angles = True, title_extension = title)
plot_phases_matrix(phases_matrix, angle_vars, align_to, cmap = 'twilight_shifted', sort_angles = True, title_extension = title)

In [ ]:
align_to = 'L1C_flex'
plot_phase_counts(phase_dict, align_to)
get_angles_aligned(phase_dict, angle_dict, align_to, margin=0.1)

In [ ]:
# phases and angles for a single fly
dist = 20
min_length = 0
align_to = 'R1C_flex'
legs = ['L1', 'R1']
fly_data = data[data['flyid'] == '4_0 5222019']
angle_vars = [v for v in fly_data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]
angle_titles = ['flexion', 'abduction', 'rotation']
cycle_dict, length_dict = get_cycles(fly_data, angle_vars, align_to = align_to, get_all_cycles = True, min_length = min_length, dist = dist)
plot_cycles(cycle_dict, length_dict, angle_vars, average = False, align_start = True, stretch = False)

align_to = 'L1C_flex'
phase_dict, angle_dict = get_phases(cycle_dict, length_dict)
plot_phases(phase_dict, angle_dict)
plot_phases_and_angles(phase_dict, angle_dict, align_to)

In [ ]:
# overlay grooming cycles from individual bouts, then determine an average grooming
# cycle for each angle (individual flies, uses all oscillations)
dist = 20
min_length = 0
align_to = 'L1C_flex'
legs = ['L1', 'R1']
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]

fly_data, fly_names = data_per_fly(data)
for j in range(len(fly_names)):
    fly_data = data[data['flyid'] == fly_names[j]]
    cycle_dict, length_dict = get_cycles(fly_data, angle_vars, align_to = align_to, get_all_cycles = True, min_length = min_length, dist = dist)
    plot_cycles(cycle_dict, length_dict, angle_vars, average = False, align_start = True, stretch = True, title_extension = '(fly ' + fly_names[j] + ')')

In [ ]:
# phases and angles for individual flies
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]
dist = 20
min_length = 0
align_to = 'L1C_flex'
fly_names = ['4_0 5222019']
for j in range(len(fly_names)):
    fly_data = data[data['flyid'] == fly_names[j]]
    cycle_dict, length_dict = get_cycles(fly_data, angle_vars, align_to = align_to, get_all_cycles = True, min_length = min_length, dist = dist)   
    phase_dict, angle_dict = get_phases(cycle_dict, length_dict)
    plot_phases(phase_dict, angle_dict)
    plot_phases_and_angles(phase_dict, angle_dict, align_to)

###### individual bouts

In [ ]:
# overlay grooming cycles from individual bouts, then determine an average grooming
# cycle for each angle (individual bouts, uses all oscillations in all bouts)
dist = 20
min_length = 0
legs = ['L1', 'R1']
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]

bout_numbers = np.unique(data.behavior_bout.astype(int))
for j in range(len(bout_numbers)):
    bout_data = data[data['behavior_bout'] == bout_numbers[j]]
    fly_id = bout_data.flyid.iloc[0]
    cycle_dict, length_dict = get_cycles(bout_data, angle_vars, get_all_cycles = True, min_length = min_length, dist = dist)
    plot_cycles(cycle_dict, length_dict, angle_vars, average = True, align_start = True, stretch = True, title_extension = '(fly ' + fly_id + ', bout ' + str(bout_numbers[j]) + ')')

In [ ]:
fly_names_sorted[:13]

In [ ]:
# phases and angles for all bouts
align_to = 'L1C_flex'
for j in range(len(bout_numbers)):
    bout_data = data[data['behavior_bout'] == bout_numbers[j]]
    fly_id = bout_data.flyid.iloc[0]
    cycle_dict, length_dict = get_cycles(bout_data, angle_vars, get_all_cycles = True, min_length = min_length, dist = dist)    
    phase_dict, angle_dict = get_phases(cycle_dict, length_dict)
    plot_phases(phase_dict, angle_dict)
    plot_phases_and_angles(phase_dict, angle_dict, align_to)

In [ ]:
align_to = 'R1C_flex'
phase_dict
sns.set()
sns.set_style('ticks')
fig = plt.figure(figsize = (5,5))
ax = plt.gca()
ax.title('average phase offsets relative to {}'.format(align_to), fontsize = 14)
ax.set_xlabel(, fontsize = 14)
ax.set_ylabel(, fontsize = 14)
plt.show()

In [ ]:
# phases and angles for one bout
dist = 20
min_length = 0
legs = ['L1']
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]
bout_numbers = np.array([105])
for j in range(len(bout_numbers)):
    bout_data = data[data['behavior_bout'] == bout_numbers[j]]
    fly_id = bout_data.flyid.iloc[0]
    cycle_dict, length_dict = get_cycles(bout_data, angle_vars, get_all_cycles = True, min_length = min_length, dist = dist)    
    phase_dict, angle_dict = get_phases(cycle_dict, length_dict)
    plot_cycles(cycle_dict, length_dict, angle_vars, average = True, align_start = True, stretch = True, title_extension = '(fly ' + fly_id + ', bout ' + str(bout_numbers[j]) + ')')

In [ ]:
# phases and angles for one bout
dist = 20
min_length = 0
legs = ['L1', 'R1']
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]
bout_numbers = np.array([105])
for j in range(len(bout_numbers)):
    bout_data = data[data['behavior_bout'] == bout_numbers[j]]
    fly_id = bout_data.flyid.iloc[0]
    cycle_dict, length_dict = get_cycles(bout_data, angle_vars, get_all_cycles = True, min_length = min_length, dist = dist)    
    phase_dict, angle_dict = get_phases(cycle_dict, length_dict)
    plot_cycles(cycle_dict, length_dict, angle_vars, average = True, align_start = True, stretch = True, title_extension = '(fly ' + fly_id + ', bout ' + str(bout_numbers[j]) + ')')
    # xs, avg, avg_data = get_average(cycle_dict['L1C_flex'], length_dict['L1C_flex'], 101)
    # plt.plot(xs, avg)

###### average cycles for individual flies

In [ ]:
# compute the average trace for each fly for each angle
# maps angle to average angle cycles for each fly
dist = 20
min_length = 0
align_to ='R1B_flex'
legs = ['L1', 'R1']
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]
fly_names = np.unique(data.flyid)
npoints = 50

average_dict = dict()
average_length_dict = dict()
xs_dict = dict()
    
for i in range(len(angle_vars)):
    
    averages = []
    xs = []
    lengths = []
    for j in range(len(fly_names)):
        fly_data = data[data['flyid'] == fly_names[j]]
        cycle_dict, length_dict = get_cycles(fly_data, angle_vars, align_to = align_to, get_all_cycles = True, min_length = min_length, dist = dist)
        if len(length_dict[angle_vars[i]]) < 1: 
            continue
        xs_all, average, avg_data = get_average(cycle_dict[angle_vars[i]], length_dict[angle_vars[i]], npoints, align_start = True, stretch = True)
        averages.append(average)
        xs.append(xs_all)
        lengths.append(np.nanmean(length_dict[angle_vars[i]]))

    average_dict[angle_vars[i]] = averages
    average_length_dict[angle_vars[i]] = lengths
    xs_dict[angle_vars[i]] = xs

In [ ]:
#plots average trace
align_start = True

for i in range(len(angle_vars)):
    
    cmap = plt.get_cmap('plasma')
    cnorm  = cx.Normalize(vmin=np.nanmin(list(average_length_dict.values())), vmax=np.nanmax(list(average_length_dict.values())))
    scalar_map = cm.ScalarMappable(norm=cnorm, cmap=cmap)
    
    fig = plt.figure(figsize = (8,4), dpi = 100)
    plt.title('average {} {} cycle'.format(angle_vars[i][:2], titles[angle_vars[i][2:]]), fontsize = 14)
    plt.xlabel('time (seconds)', fontsize = 14)
    plt.ylabel('angle (deg)', fontsize = 14)
    plt.xticks([])
    averages = np.array(average_dict[angle_vars[i]])
    xs = np.array(xs_dict[angle_vars[i]])
    lengths = average_length_dict[angle_vars[i]]
    for j in range(averages.shape[1]):
        avg = averages[j, :]
        if align_start:
            avg = avg - avg[0]
        plt.plot(xs[j, :], avg, color = scalar_map.to_rgba(lengths[j]), linewidth = 1)
    average_cycle = np.nanmean(averages, axis = 0)
    if align_start:
        average_cycle = average_cycle-average_cycle[0]
    plt.plot(xs.T, average_cycle, 'k', linewidth = 2)
    figure_path = '/media/turritopsis/katie/grooming/grooming_cycles'
    plt.tight_layout()
    plt.savefig(os.path.join(figure_path, angle_vars[i] + ' average cycle (fly averages)'))
    plt.show()

In [ ]:
# phases and angles for all flies 
align_to = 'L1C_flex'
npoints = 50
phase_dict, angle_dict = get_phases(cycle_dict, length_dict, npoints)
subplot_phases(phase_dict, angle_dict)
subplot_phases_and_angles(phase_dict, angle_dict, align_to)

###### variance across flies

In [ ]:
# compute the RMSE for each angle from the average angle cycle (average)
rmse_dict = dict()
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]
for i in range(len(angle_vars)):
    rmses = []
    averages = np.array(average_dict[angle_vars[i]])
    average_cycle = np.nanmean(averages, axis = 0)
    for j in range(len(averages)):
        # var = np.mean(np.var([averages[j, :], average_cycle], axis = 0))
        error = rmse(averages[j, :], average_cycle)
        rmses.append(error)
    rmse_dict[angle_vars[i]] = rmses

In [ ]:
# plot variance for each angle type (scatter with means)
legs = ['L1', 'R1']
angle_types = ['_flex', '_abduct', '_rot']
angle_titles = ['flexion', 'abduction', 'rotation']
ylims = [80, 35, 100]
for i in range(len(angle_types)):
    angle_vars = [v for v in data.columns
                  if some_contains(v, [angle_types[i]])
                  and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
                  and v[:2] in legs]
    fig = plt.figure(figsize = (8,4))
    plt.title('error of an average cycle of a fly to the average cycle of all flies'.format(angle_titles[i]), fontsize = 14)
    plt.xlabel('joints', fontsize = 14)
    plt.xticks(ticks = np.arange(0.5, (len(angle_vars)), 1), labels = angle_vars)
    plt.ylabel('RMSE (degrees)', fontsize = 14)
    ax = plt.gca()
    plt.ylim([0, ylims[i]])
    
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    n = len(angle_vars)
    pos = np.arange(0.5, n + 0.5, 1)
    for j in range(len(angle_vars)):
        rmses = np.array(rmse_dict[angle_vars[j]])
        rmses = rmses[rmses < np.percentile(rmses, 95)]
        avg_rmse = np.nanmean(rmses)
        stderr_rmse = stats.sem(rmses)
        ax.errorbar(pos[j], avg_rmse, yerr = stderr_rmse, fmt = 'none', capsize = 7)
        k = j + 1
        sc = 0.5*np.random.rand(rmses.shape[0]) - 0.75
        plt.scatter(np.ones(rmses.shape[0]) + j + sc, rmses, s = 2, color = colors[j])
        x_b = j + 0.5
        x_t = (j + 1)
        ax.axhline(y = avg_rmse, xmin = j/n + 0.02, xmax = k/n - 0.02, color = colors[j])
        plt.xlim([0,n])
    
    sns.despine()
    plt.tight_layout()
    plt.show()

###### variance within a fly

In [ ]:
# compute the variance within a fly for each angle from the average angle cycle (average)
dist = 20
min_length = 0
align_to = 'R1B_flex'
rmse_dict = dict()
fly_names = np.unique(data.flyid)
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]
for i in range(len(angle_vars)):
    
    rmse_flies = []
    for j in range(len(fly_names)):
    
        fly_data = data[data.flyid == fly_names[j]]
        cycle_dict, length_dict = get_cycles(fly_data, angle_vars, align_to = align_to, get_all_cycles = True, min_length = min_length, dist = dist)
        all_lengths = np.array(length_dict[angle_vars[i]])
        if len(all_lengths) < 1: 
            continue
        all_cycles = np.array(cycle_dict[angle_vars[i]]) 
        xs_all, avg, bouts = get_average(all_cycles, all_lengths, 50, align_start = True, stretch = True)
        average_cycle = np.nanmean(bouts, axis = 0)
        rmse_bouts = []
        for k in range(len(bouts)):
            
            # var = np.mean(np.var([bouts[k], average_cycle], axis = 0))
            error = rmse(bouts[k], average_cycle)
            rmse_bouts.append(error)
            
        rmse_flies.append(np.nanmean(rmse_bouts))
        
    rmse_dict[angle_vars[i]] = rmse_flies

In [ ]:
# plot average rmse within a fly for each angle type (scatter with means)
legs = ['L1', 'R1']
angle_types = ['_flex', '_abduct', '_rot']
angle_titles = ['flexion', 'abduction', 'rotation']
ylims = [80, 35, 100]
for i in range(len(angle_types)):
    angle_vars = [v for v in data.columns
                  if some_contains(v, [angle_types[i]])
                  and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
                  and v[:2] in legs]
    fig = plt.figure(figsize = (8,4))
    plt.title('error of a cycle of a fly to the average cycle of the fly'.format(angle_titles[i]), fontsize = 14)
    plt.xlabel('joints', fontsize = 14)
    plt.xticks(ticks = np.arange(0.5, (len(angle_vars)), 1), labels = angle_vars)
    plt.ylabel('RMSE (degrees)', fontsize = 14)
    ax = plt.gca()
    plt.ylim([0, ylims[i]])
    
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    n = len(angle_vars)
    pos = np.arange(0.5, n + 0.5, 1)
    for j in range(len(angle_vars)):
        rmses = np.array(rmse_dict[angle_vars[j]])
        # rmses = rmses[rmses < np.percentile(rmses, 97)]
        avg_rmse = np.nanmean(rmses)
        stderr_rmse = stats.sem(rmses)
        ax.errorbar(pos[j], avg_rmse, yerr = stderr_rmse, fmt = 'none', capsize = 5)
        k = j + 1
        sc = 0.5*np.random.rand(rmses.shape[0]) - 0.75
        plt.scatter(np.ones(rmses.shape[0]) + j + sc, rmses, s = 2, color = colors[j])
        x_b = j + 0.5
        x_t = (j + 1)
        ax.axhline(y = avg_rmse, xmin = j/n + 0.02, xmax = k/n - 0.02, color = colors[j])
        plt.xlim([0,n])
    
    sns.despine()
    plt.tight_layout()
    plt.show()

###### variance across time

In [ ]:
# compute the variance across time for each angle from the average angle cycle (average)
angle_vars = [v for v in data.columns
              if some_contains(v, ['_flex', '_abduct', '_rot'])
              and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
              and v[:2] in legs]
time_var_dict = dict()
for i in range(len(angle_vars)):
    # variance = []
    rmses = []
    averages = np.array(average_dict[angle_vars[i]])
    average_cycle = np.nanmean(averages, axis = 0)
    for j in range(averages.shape[1]):
        #var = np.var([averages[j, :], average_cycle], axis = 0)
        #variance.append(var)
        rmses.append(rmse(averages[:, j], [average_cycle[j]]*len(averages[:,j])))
    # time_var_dict[angle_vars[i]] = variance
    time_var_dict[angle_vars[i]] = rmses

In [ ]:
average_dict
average_cycle
[averages[j, :], average_cycle]
rmse(averages[:, 0], [average_cycle[0]]*len(averages[:,0]))

In [ ]:
# plot variance across time (scatter with means)
legs = ['L1', 'R1']
angle_types = ['_flex', '_abduct', '_rot']
angle_titles = ['flexion', 'abduction', 'rotation']
for i in range(len(angle_types)):
    angle_vars = [v for v in data.columns
                  if some_contains(v, [angle_types[i]])
                  and not some_contains(v, ['_d1', '_d2', '_freq', '_range'])
                  and v[:2] in legs]
    fig = plt.figure(figsize = (8,4))
    plt.title('RMSE of {} angle cycles across time'.format(angle_titles[i]), fontsize = 14)
    plt.xlabel('time', fontsize = 14)
    plt.ylabel('RMSE', fontsize = 14)
    plt.xticks([])
    plt.ylim([0, 14])
    ax = plt.gca()
    
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    n = len(angle_vars)
    pos = np.arange(0.5, n + 0.5, 1)
    for j in range(len(angle_vars)):
        time_var = np.array(time_var_dict[angle_vars[j]])
        var = time_var # np.nanmean(time_var, axis = 0)
        plt.plot(range(len(var)), var, label = angle_vars[j])
    
    sns.despine()
    plt.tight_layout()
    # plt.legend(loc = (1, 0.5))
    plt.show()

In [ ]:
align_start = True
dist = 15
npoints = 100
average_dict = dict()
xs_dict = dict()
average_length_dict = dict()
fly_names = np.unique(data.flyid)

for j in range(len(fly_names)):
    averages = []
    xs = []
    average_lengths = []
    fly_data = data[data['flyid'] == fly_names[j]]
    cycle_dict, length_dict = get_cycles(fly_data, angle_vars, align_to = align_to, get_all_cycles = True, min_length = min_length, dist = dist)
        
    for i in range(len(angle_vars)):
        if len(length_dict[angle_vars[i]]) < 4:
            averages.append([])
            xs.append([])
            continue
        xs_all, average, avg_data = get_average(cycle_dict[angle_vars[i]], length_dict[angle_vars[i]], npoints, align_start = True, stretch = True)
        if align_start:
            average = average - average[0]
        averages.append(average)
        xs.append(xs_all)
        
    average_dict[fly_names[j]] = averages
    xs_dict[fly_names[j]] = xs
    average_length_dict[fly_names[j]] = np.nanmean(length_dict[angle_vars[i]])

In [ ]:
for i in range(len(angle_vars)):
    
    cmap = plt.get_cmap('plasma')
    cnorm  = cx.Normalize(vmin=np.nanmin(list(average_length_dict.values())), vmax=np.nanmax(list(average_length_dict.values())))
    scalar_map = cm.ScalarMappable(norm=cnorm, cmap=cmap)
    
    fig = plt.figure(figsize = (8,4))
    plt.title('average {} {} cycle'.format(angle_vars[i][:2], titles[angle_vars[i][2:]]), fontsize = 14)
    plt.xlabel('time (seconds)', fontsize = 14)
    plt.ylabel('angle (deg)', fontsize = 14)
    plt.xticks([])
    average_cycles = []
    variance = []
    for j in range(len(fly_names)):
        averages = average_dict[fly_names[j]]
        if len(averages[i]) < 1:
            continue
        average_cycles.append(averages[i])
        xs = xs_dict[fly_names[j]]
        plt.plot(xs[i], averages[i], color = scalar_map.to_rgba(average_length_dict[fly_names[j]]))
    average_cycle = np.nanmean(np.array(average_cycles), axis = 0)
    plt.plot(xs[i], average_cycle, 'k', linewidth = 2)
    figure_path = '/media/turritopsis/katie/grooming/grooming_cycles'
    plt.tight_layout()
    plt.savefig(os.path.join(figure_path, angle_vars[i] + ' average cycle (fly averages)'))
    plt.show()